<a href="https://colab.research.google.com/github/santoshpremi/Perceptual_Learned_Image_Compression/blob/main/Colab_PLIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Perceptual Learned Image Compression
Introduction:
With most of the internet traffic being image or (esp.) video data, image compression algorithms
are essential to being able to handle this large amount of data. Neural Image Compression (NIC)
methods have been shown to achieve significantly better compression rates compared to
handcrafted algorithms [1]. However, both approaches can suffer from artifacts or overly

---


smoothed images.
Perceptual Image Compression [2, 3] tries to generate reconstructions that are perceived to be of
high quality, even if the actual pixel-values might differ from the original image.
Goals:
Design and implement a NIC method based on a SOTA method
Explore different approaches for optimizing for perceptual quality
Evaluate on benchmark datasets in terms of distortion, perceptual quality and speed
Desirable Experience:
Knowledge of Computer Vision, Deep Learning and Compression
Experience in the PyTorch or TensorFlow framework

References
1. He, Dailan, Ziming Yang, Hongjiu Yu, Tongda Xu, Jixiang Luo, Yuan Chen, Chenjian Gao, Xinjie Shi,
Hongwei Qin, and Yan Wang. “PO-ELIC: Perception-Oriented Efficient Learned Image Coding.” In
2022 IEEE/CVF Conference on Computer Vision and Pattern Recognition Workshops (CVPRW), 1763–
68. New Orleans, LA, USA: IEEE, 2022. https://doi.org/10.1109/CVPRW56347.2022.00187.
2. Mentzer, Fabian, George Toderici, Michael Tschannen, and Eirikur Agustsson. “High-Fidelity
Generative Image Compression.” arXiv, October 23, 2020.
https://doi.org/10.48550/arXiv.2006.09965.
3. Ning, Peirong, Wei Jiang, and Ronggang Wang. “HFLIC: Human Friendly Perceptual Learned Image
Compression with Reinforced Transform.” arXiv, May 18, 2023.
https://doi.org/10.48550/arXiv.2305.07519.





## HFLIC: Human Friendly Perceptual Learned Image Compression with Reinforced Transform

This notebook implements **HFLIC (Human Friendly Perceptual Learned Image Compression)** based on the paper "HFLIC: Human Friendly Perceptual Learned Image Compression with Reinforced Transform" by extending ELIC with reinforced transform blocks for better perceptual quality.

In [ ]:
import pathlib
import sys
import subprocess

REPO_URL = "https://github.com/santoshpremi/Perceptual_Learned_Image_Compression.git"
PROJECT_DIR = pathlib.Path("/content/Perceptual_Learned_Image_Compression")

if PROJECT_DIR.exists():
    print(f"Repository already present at {PROJECT_DIR}, pulling latest changes...")
    subprocess.run(["git", "-C", str(PROJECT_DIR), "pull"], check=False)
else:
    print(f"Cloning repository from {REPO_URL} to {PROJECT_DIR}...")
    subprocess.run(["git", "clone", REPO_URL, str(PROJECT_DIR)], check=True)

%pip install -q -r {PROJECT_DIR}/requirements.txt
%pip install -q lpips

sys.path.append(str(PROJECT_DIR))
print("Environment setup complete.")

## **Train HFLIC: Vimeo90K dataset and save weights**

Training HFLIC model on Vimeo90K dataset with perceptual loss:
- **Loss = 0.1·MSE + 0.9·LPIPS + 10.0·λ·BPP**
- Model: HFLIC with N=192, M=320
- Training on Vimeo90K test split (for fine-tuning)
- Saving checkpoints for evaluation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


!ls "/content"

Mounted at /content/drive
drive  sample_data


In [ ]:
import pathlib

PROJECT_DIR = pathlib.Path("/content/Perceptual_Learned_Image_Compression")
assert PROJECT_DIR.exists(), "Run the setup cell first to clone the repository."

# Point these to your Google Drive datasets
TRAIN_ROOT = "/content/drive/MyDrive/vimeo_test_clean"
EVAL_ROOT = "/content/drive/MyDrive"

print("Launching HFLIC training from the project scripts...")
!cd {PROJECT_DIR} && \
 python train_gan.py \
   --train-root {TRAIN_ROOT} \
   --train-list sep_testlist.txt \
   --train-frame-index 4 \
   --eval-root {EVAL_ROOT} \
   --eval-split kodak \
   --epochs 10 \
   --batch-size 4 \
   --cuda \
   --experiment colab_run



## **Evaluating HFLIC: Kodak dataset with saved weights**

Evaluating trained HFLIC model on Kodak dataset (24 standard test images) with comprehensive metrics:
- **PSNR**: Peak Signal-to-Noise Ratio (dB)
- **LPIPS**: Learned Perceptual Image Patch Similarity (lower is better)
- **BPP**: Bits Per Pixel (compression efficiency)
- **MSE**: Mean Squared Error

---



In [ ]:
import pathlib

PROJECT_DIR = pathlib.Path("/content/Perceptual_Learned_Image_Compression")
assert PROJECT_DIR.exists(), "Run the setup cell first to clone the repository."

# Update these paths before running the cell
CHECKPOINT_PATH = "/content/Perceptual_Learned_Image_Compression/experiments/colab_run/checkpoints/checkpoint_010.pth.tar"
DATA_ROOT = "/content/drive/MyDrive"

print("Evaluating HFLIC checkpoint on Kodak...")
!python {PROJECT_DIR}/test_5.py \
  --dataset {DATA_ROOT} \
  --split kodak \
  --checkpoint {CHECKPOINT_PATH} \
  --test-batch-size 1 \
  --cuda